In [6]:
import cv2 as cv
import mediapipe as mp
import pandas as pd  
import os
import numpy as np 
import requests as reqs 
import webbrowser
from PIL import ImageFont, ImageDraw, Image

In [7]:
font_path = r"C:\Users\hfhon\Downloads\phetsarath_ot.ttf"  # Provide the path to a font file that supports Lao characters
font_size = 105
font = ImageFont.truetype(font_path, font_size)

In [8]:
def image_processed(hand_img):

    # Image processing
    # 1. Convert BGR to RGB
    img_rgb = cv.cvtColor(hand_img, cv.COLOR_BGR2RGB)

    # 2. Flip the img in Y-axis
    img_flip = cv.flip(img_rgb, 1)

    # accessing MediaPipe solutions
    mp_hands = mp.solutions.hands

    # Initialize Hands
    hands = mp_hands.Hands(static_image_mode=True,
    max_num_hands=1, min_detection_confidence=0.7)

    # Results
    output = hands.process(img_flip)

    hands.close()

    try:
        data = output.multi_hand_landmarks[0]
        #print(data)
        data = str(data)

        data = data.strip().split('\n')

        garbage = ['landmark {', '  visibility: 0.0', '  presence: 0.0', '}']

        without_garbage = []

        for i in data:
            if i not in garbage:
                without_garbage.append(i)
                        
        clean = []

        for i in without_garbage:
            i = i.strip()
            clean.append(i[2:])

        for i in range(0, len(clean)):
            clean[i] = float(clean[i])
        return(clean)
    except:
        return(np.zeros([1,63], dtype=int)[0])

In [9]:
import pickle
# load model
with open('model_mckiler.pkl', 'rb') as f:
    svm = pickle.load(f)

In [11]:

cap = cv.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    ret, frame = cap.read()

    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # Process the frame
    data = image_processed(frame)

    # Convert data to a NumPy array before reshaping
    data = np.array(data)
    
    # Perform prediction and obtain y_pred
    y_pred = svm.predict(data.reshape(-1, 63))
    
    # Convert the prediction to string (if necessary)
    y_pred_str = str(y_pred[0])  # Convert to string if y_pred is not already a string
    
    # Draw Lao characters onto the image
    pil_image = Image.fromarray(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_image)
    org = (50, 100)
    color = (255, 0, 0)
    text = y_pred_str  # Use the predicted string directly
    draw.text(org, text, font=font, fill=color)

    # Convert back to OpenCV format and draw on frame
    frame_with_text = cv.cvtColor(np.array(pil_image), cv.COLOR_RGB2BGR)
    #frame_with_text = cv.putText(frame_with_text, text, org, cv.FONT_HERSHEY_SIMPLEX, fontScale=1, color=color, thickness=2)

    # Display the frame with prediction
    cv.imshow('frame', frame_with_text)

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

C:\Users\hfhon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\hfhon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\hfhon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\protobuf\symbol_database.py:

KeyboardInterrupt: 

: 